In [0]:
# dbutils.fs.rm(
#     "/Volumes/workspace/stream/streaming/jsonSink/data/",
#     recurse=True
# )
# dbutils.fs.rm(
#     "/Volumes/workspace/stream/streaming/jsonSink/archive/",
#     recurse=True
# )
# dbutils.fs.rm(
#     "/Volumes/workspace/stream/streaming/jsonSink/checkpoint/",
#     recurse=True
# )
# dbutils.fs.rm(
#     "/Volumes/workspace/stream/streaming/jsonSource/",
#     recurse=True
# )

# dbutils.fs.mkdirs("/Volumes/workspace/stream/streaming/jsonSink/data/")
# dbutils.fs.mkdirs("/Volumes/workspace/stream/streaming/jsonSink/archive/")
# dbutils.fs.mkdirs("/Volumes/workspace/stream/streaming/jsonSink/checkpoint/")
# dbutils.fs.mkdirs("/Volumes/workspace/stream/streaming/jsonSource/")

### Read JSON data (BATCH)

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
# df = spark.read.format("json") \
#     .option("multiline", True) \
#     .load("/Volumes/workspace/stream/streaming/jsonSource/day1.json")

In [0]:
# df.display()

In [0]:
# df = df.withColumn("items", explode_outer("items")) \
#        .withColumn("metadata", explode_outer("metadata"))
# df.display()

In [0]:
# df = df.select("customer.customer_id","customer.email","customer.name","customer.address.city","customer.address.country","customer.address.postal_code","items.item_id","items.price","items.product_name","items.quantity","metadata.key","metadata.value", "order_id","payment.method","payment.transaction_id","timestamp")
# df.display()

### READ STREAMING DATA

In [0]:
from pyspark.sql.types import *

# sch = StructType([
#     StructField("customer", StructType([
#         StructField("customer_id", IntegerType()),
#         StructField("email", StringType()),
#         StructField("name", StringType()),
#         StructField("address", StructType([
#             StructField("city", StringType()),
#             StructField("country", StringType()),
#             StructField("postal_code", StringType())
#         ]))
#     ])),
#     StructField("items", ArrayType(StructType([
#         StructField("item_id", StringType()),
#         StructField("price", DoubleType()),
#         StructField("product_name", StringType()),
#         StructField("quantity", LongType())
#     ]))),
#     StructField("metadata", ArrayType(StructType([
#         StructField("key", StringType()),
#         StructField("value", StringType())
#     ]))),
#     StructField("order_id", StringType()),
#     StructField("payment", StructType([
#         StructField("method", StringType()),
#         StructField("transaction_id", StringType())
#     ])),
#     StructField("timestamp", StringType())
# ])

sch = StructType([
    StructField("city", StringType()),
    StructField("temperature", FloatType()),
    StructField("humidity", IntegerType()),
    StructField("timestamp", DateType())
])

In [0]:
df2 = spark.readStream.format("json") \
    .option("multiline", True) \
    .schema(sch) \
    .option("cleanSource", "archive") \
    .option("sourceArchiveDir", "/Volumes/workspace/stream/streaming/jsonSink/archive") \
    .load("/Volumes/workspace/stream/streaming/jsonSource")

# df2 = df2.withColumn("items", explode_outer("items")) \
#          .withColumn("metadata", explode_outer("metadata"))

df2 = df2.select("city","temperature","humidity","timestamp")

# "customer.customer_id", "customer.email", "customer.name",
# "customer.address.city", "customer.address.country", "customer.address.postal_code",
# "items.item_id", "items.price", "items.product_name", "items.quantity",
# "metadata.key", "metadata.value", "order_id",
# "payment.method", "payment.transaction_id", "timestamp"
# )

df2.writeStream.format("delta") \
    .outputMode("append") \
    .trigger(once=True) \
    .option("path", "/Volumes/workspace/stream/streaming/jsonSink/data") \
    .option("checkpointLocation", "/Volumes/workspace/stream/streaming/jsonSink/checkpoint") \
    .start()


In [0]:
%sql
SELECT * FROM delta.`/Volumes/workspace/stream/streaming/jsonSink/data/`